# FRENCHTOP 20 QULAC

First start with english dataset just to set up the runs, do it on top 20 docs. For now we dont put any parameters, we just use the docs and the query and try to predict 1 claridying query.

In [ ]:
pip install datasets


In [6]:
def get_token_length(input_text):
    tokenized_input = tokenizer(input_text, padding=False, truncation=False)
    return len(tokenized_input["input_ids"])

# Calculate the average token length for all inputs
token_lengths = [get_token_length(entry["input"]) for entry in raw_data]

# Calculate the average and print it
average_token_length = sum(token_lengths) / len(token_lengths)
print(f"Average token length: {average_token_length}")

Average token length: 86448.01502504174


In [ ]:
pip install wandb

In [1]:
import wandb

# Paste your API key directly here
wandb.login(key="")


wandb: ERROR Failed to detect the name of this notebook. You can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /users/Etu0/21402600/.netrc
wandb: Currently logged in as: hibaakbi (hibaakbi-sorbonne-universit-) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [ ]:
pip install evaluate

In [ ]:
pip install rouge_score

In [2]:
from huggingface_hub import login

login("")

In [3]:
from evaluate import load

bleu = load("bleu")
rouge = load("rouge")
meteor = load("meteor")

2025-05-13 02:08:18.813475: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747094898.835604   62038 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747094898.842502   62038 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1747094898.859634   62038 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1747094898.859655   62038 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1747094898.859658   62038 computation_placer.cc:177] computation placer alr

In [ ]:
# from transformers import AutoModelForSeq2SeqLM, Trainer, TrainingArguments,EarlyStoppingCallback
# from datasets import Dataset
# import os


# model_name = "t5-small"
# model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# output_dir = "./results"
# logging_dir = os.path.join(output_dir, "logs")
# # Define training arguments
# training_args = TrainingArguments(
#     output_dir="./results",  # Output directory for logs and checkpoints (optional)
#     eval_strategy="epoch",  # Evaluate after every epoch
#     save_strategy="epoch",
#     learning_rate=5e-5,  # Learning rate
#     per_device_train_batch_size=16,  # Batch size per device for training
#     per_device_eval_batch_size=16,  # Batch size per device for evaluation
#     num_train_epochs=30,  # Number of training epochs
#     logging_dir=logging_dir,
#     use_cpu=True,
#     logging_first_step=True,
#     weight_decay=0.01,  # Weight decay for regularization
#     logging_steps=50,
#     load_best_model_at_end=True,
#     metric_for_best_model="eval_loss", 
#     greater_is_better=False
# )

# os.makedirs(logging_dir, exist_ok=True)

# # Initialize the Trainer
# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=tokenized_train_dataset,
#     eval_dataset=tokenized_eval_dataset,
#     compute_metrics=compute_metrics,
#     tokenizer=tokenizer,  # Pass the tokenizer for automatic tokenization during training
#     callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
# )

# # Start the training (without saving)
# trainer.train()

# # You can evaluate the model here if you need
# # Example: Evaluate after training
# evaluation_results = trainer.evaluate()
# print(evaluation_results)



In [44]:
# predictions = trainer.predict(tokenized_eval_dataset)

# pred_ids = np.argmax(predictions.predictions[0], axis=-1)
# predicted_texts = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
# true_texts = tokenizer.batch_decode(predictions.label_ids, skip_special_tokens=True)

# output_predictions_file = "predicted_clarif_TINYT5_30EPOCH.jsonl"
# with open(output_predictions_file, "w", encoding="utf-8") as writer:
#     for true_text, predicted_text in zip(true_texts, predicted_texts):
#         output_dict = {"true": true_text, "predicted": predicted_text}
#         writer.write(json.dumps(output_dict) + "\n")

# print(f"Final predictions saved to {output_predictions_file}")

Final predictions saved to predicted_clarif_TINYT5_30EPOCH.jsonl


# -----------------------------------------------------------------

# Cross validation

# T5 EFFICIENT -------------- TOP5 RAW DOCS 

In [6]:
from sklearn.model_selection import KFold
from datasets import Dataset
from transformers import AutoTokenizer
import json
import numpy as np

#fucntion to tokenize the dataset based on the model
def tokenize_and_split_dataset(filename, modelname):
    with open(filename, "r", encoding="utf-8") as f:
        raw_data = json.load(f)
        
    # Choose your model
    tokenizer = AutoTokenizer.from_pretrained(modelname)

    # Tokenization function
    def preprocess(batch):
        model_inputs = tokenizer(
            batch["input"],
            max_length=512,
            padding="max_length",
            truncation=True
        )
        with tokenizer.as_target_tokenizer():
            labels = tokenizer(
                batch["output"],
                max_length=64,
                padding="max_length",
                truncation=True
            )
        model_inputs["labels"] = labels["input_ids"]
        return model_inputs
    
    # Convert raw data to Dataset object
    dataset = Dataset.from_list(raw_data)
    tokenized_dataset = dataset.map(preprocess, batched=True)

    return tokenizer, tokenized_dataset


In [7]:
from sklearn.model_selection import KFold

#fucntion to split dataset into folds
def generate_folds(tokenized_dataset, n_splits=5, seed=42):
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=seed)
    folds = list(kf.split(tokenized_dataset))
    return folds


In [8]:
import wandb
from transformers import AutoModelForSeq2SeqLM, Trainer, TrainingArguments, EarlyStoppingCallback, TrainerCallback
from datasets import Dataset
import numpy as np
import json
import os
from tqdm import tqdm

# Custom callback to log metrics after each evaluation step
class WandbLoggingCallback(TrainerCallback):
    def __init__(self, fold_num):
        self.fold_num = fold_num

    def on_evaluate(self, args, state, control, metrics, **kwargs):
        if state.is_world_process_zero:
            log_data = {f"Fold_{self.fold_num}/eval/{key}": value for key, value in metrics.items()}
            wandb.log(log_data)

def cross_val_train(modelname, filename, run_name, tokenizer, tokenized_dataset, folds, nb_epochs=30):
    # Start a single WandB run for all folds
    wandb.init(project="cross_val_T5", name=run_name, config={"epochs": nb_epochs})
    filenamebis= filename.split(".jsonl")[0]
    fold_metrics = []

    for fold_num, (train_idx, test_idx) in enumerate(folds):
        print(f"Processing Fold {fold_num + 1}")

        # Split the dataset
        train_dataset = tokenized_dataset.select(train_idx)
        test_dataset = tokenized_dataset.select(test_idx)

        # Initialize the model
        model = AutoModelForSeq2SeqLM.from_pretrained(modelname, from_tf=True)

        # Training arguments with early stopping
        training_args = TrainingArguments(
            output_dir=f"./results/{run_name}/fold_{fold_num+1}",
            eval_strategy="epoch",
            save_strategy="epoch",
            learning_rate=5e-5,
            per_device_train_batch_size=8,
            per_device_eval_batch_size=8,
            num_train_epochs=nb_epochs,
            load_best_model_at_end=True,
            logging_steps=20,
            logging_first_step=True,
            metric_for_best_model="eval_loss",
            greater_is_better=False,
            seed=42,
        )

        # Initialize Trainer with the logging callback and early stopping
        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=train_dataset,
            eval_dataset=test_dataset,
            tokenizer=tokenizer,
            # compute_metrics=compute_metrics,
            compute_metrics=lambda eval_pred: compute_metrics(eval_pred, tokenizer),
            callbacks=[
                WandbLoggingCallback(fold_num + 1),
                EarlyStoppingCallback(early_stopping_patience=3)
            ],
        )

        # Train the model
        trainer.train()

        # Final evaluation for the fold
        eval_results = trainer.evaluate()
        print(f"Fold {fold_num + 1} results: {eval_results}")

        # Store the final evaluation results for this fold
        fold_metrics.append(eval_results)

        # Predict on the test dataset
        predictions = trainer.predict(test_dataset)

    
        # Decode predictions and save to file
        pred_ids = np.argmax(predictions.predictions[0], axis=-1)
        predicted_texts = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
        true_texts = tokenizer.batch_decode(predictions.label_ids, skip_special_tokens=True)
        input_ids = test_dataset["input_ids"]
        input_texts = tokenizer.batch_decode(input_ids, skip_special_tokens=True)

        # Save predictions to a file

        output_predictions_file = f"{filenamebis}_fold_{fold_num+1}.jsonl"
        with open(output_predictions_file, "w", encoding="utf-8") as writer:
            for input_text, true_text, predicted_text in tqdm(zip(input_texts, true_texts, predicted_texts), total=len(true_texts)):
                output_dict = {
                    "input": input_text,
                    "true": true_text,
                    "predicted": predicted_text
                }
                writer.write(json.dumps(output_dict, ensure_ascii=False) + "\n")
        
        print(f"Predictions for fold {fold_num + 1} saved to {output_predictions_file}")

    # Calculate and log average metrics
    avg_metrics = {key: np.mean([fold[key] for fold in fold_metrics]) for key in fold_metrics[0]}
    wandb.log({f"Averages/{key}": value for key, value in avg_metrics.items()})
    print("Average metrics over all folds:", avg_metrics)

    wandb.finish()

    #merge all to a single file for predcitions

    with open(filename, "w", encoding="utf-8") as merged_file:
        for fold_num in range(len(folds)):
            fold_file = f"{filenamebis}_fold_{fold_num+1}.jsonl"
            if os.path.exists(fold_file):
                with open(fold_file, "r", encoding="utf-8") as f:
                    for line in f:
                        merged_file.write(line)
    print(f"Final predicitons for all folds foudn in {filename}")


In [ ]:
#WORKING VERSION OF CV WITHOUT PROPER LOGGING

# import wandb
# from transformers import AutoModelForSeq2SeqLM, Trainer, TrainingArguments, EarlyStoppingCallback
# from datasets import Dataset
# import os
# from tqdm import tqdm
# from sklearn.model_selection import KFold
# import numpy as np

# def cross_val_train(modelname, filename, run_name, tokenizer, tokenized_dataset, folds, nb_epochs=30):
#     # Start a single WandB run for all folds
#     wandb.init(project="cross_val_Test", name=run_name, config={"epochs": nb_epochs})

#     fold_metrics = []

#     for fold_num, (train_idx, test_idx) in enumerate(folds):
#         print(f"Processing Fold {fold_num + 1}")

#         # Split the dataset
#         train_dataset = tokenized_dataset.select(train_idx)
#         test_dataset = tokenized_dataset.select(test_idx)

#         # Initialize the model for each fold
#         model = AutoModelForSeq2SeqLM.from_pretrained(modelname, from_tf=True)

#         # Training arguments
#         training_args = TrainingArguments(
#             output_dir=f"./results/fold_{fold_num+1}",  # Different folder per fold
#             eval_strategy="epoch",
#             save_strategy="epoch",
#             learning_rate=5e-5,
#             per_device_train_batch_size=8,
#             per_device_eval_batch_size=8,
#             num_train_epochs=nb_epochs,
#             logging_dir=f"./results/logs/fold_{fold_num+1}",
#             load_best_model_at_end=True,
#             metric_for_best_model="eval_loss",
#             greater_is_better=False,
#             seed=42,  # Seed for reproducibility
#         )

#         # Initialize Trainer
#         trainer = Trainer(
#             model=model,
#             args=training_args,
#             train_dataset=train_dataset,
#             eval_dataset=test_dataset,
#             tokenizer=tokenizer,
#             compute_metrics=compute_metrics,
#         )

#         # Train the model
#         trainer.train()

#         # Evaluate the model on this fold
#         eval_results = trainer.evaluate()
#         print(f"Fold {fold_num + 1} results: {eval_results}")

#         # Store the fold metrics
#         fold_metrics.append(eval_results)

#     # Aggregate results over all folds
#     avg_metrics = {key: np.mean([fold[key] for fold in fold_metrics]) for key in fold_metrics[0]}
#     print("Average metrics over all folds:", avg_metrics)

#     # Log average metrics to WandB

#     # End the WandB run
#     wandb.finish()


In [6]:
# import wandb
# from transformers import AutoModelForSeq2SeqLM, Trainer, TrainingArguments, EarlyStoppingCallback, TrainerCallback
# from datasets import Dataset
# import numpy as np

# # Custom callback to log metrics after each evaluation step
# class WandbLoggingCallback(TrainerCallback):
#     def __init__(self, fold_num):
#         self.fold_num = fold_num

#     def on_evaluate(self, args, state, control, metrics, **kwargs):
#         if state.is_world_process_zero:
#             # Log metrics with fold-specific keys
#             log_data = {f"Fold_{self.fold_num}/eval/{key}": value for key, value in metrics.items()}
#             wandb.log(log_data)

# def cross_val_train(modelname, filename, run_name, tokenizer, tokenized_dataset, folds, nb_epochs=30):
#     # Start a single WandB run for all folds
#     wandb.init(project="cross_val_T5", name=run_name, config={"epochs": nb_epochs})

#     fold_metrics = []

#     for fold_num, (train_idx, test_idx) in enumerate(folds):
#         print(f"Processing Fold {fold_num + 1}")

#         # Split the dataset
#         train_dataset = tokenized_dataset.select(train_idx)
#         test_dataset = tokenized_dataset.select(test_idx)

#         # Initialize the model
#         model = AutoModelForSeq2SeqLM.from_pretrained(modelname, from_tf=True)

#         # Training arguments with early stopping
#         training_args = TrainingArguments(
#             output_dir=f"./results/fold_{fold_num+1}",
#             eval_strategy="epoch",
#             save_strategy="epoch",
#             learning_rate=5e-5,
#             per_device_train_batch_size=8,
#             per_device_eval_batch_size=8,
#             num_train_epochs=nb_epochs,
#             logging_dir=f"./results/logs/fold_{fold_num+1}",
#             load_best_model_at_end=True,
#             logging_steps=20,
#             logging_first_step=True,
#             metric_for_best_model="eval_loss",
#             greater_is_better=False,
#             seed=42,
#         )

#         # Initialize Trainer with the logging callback and early stopping
#         trainer = Trainer(
#             model=model,
#             args=training_args,
#             train_dataset=train_dataset,
#             eval_dataset=test_dataset,
#             tokenizer=tokenizer,
#             compute_metrics=compute_metrics,
#             callbacks=[
#                 WandbLoggingCallback(fold_num + 1),  # Log metrics for each fold
#                 EarlyStoppingCallback(early_stopping_patience=3)  # Early stopping after 3 epochs of no improvement
#             ],
#         )

#         # Train the model
#         trainer.train()

#         # Final evaluation for the fold
#         eval_results = trainer.evaluate()
#         print(f"Fold {fold_num + 1} results: {eval_results}")

#         # Store the final evaluation results for this fold
#         fold_metrics.append(eval_results)

#     # Calculate and log average metrics
#     avg_metrics = {key: np.mean([fold[key] for fold in fold_metrics]) for key in fold_metrics[0]}
#     wandb.log({f"Averages/{key}": value for key, value in avg_metrics.items()})
#     print("Average metrics over all folds:", avg_metrics)

#     wandb.finish()


# T5 SMALL ------------------- TO5 RAW DOCS

In [51]:
tokenizer,tokenized_dataset= tokenize_and_split_dataset("training_top5_qulac_PREPROCESSED_FOR_MODEL.json","t5-small")
print(tokenized_dataset[0])  # Optional: check one example

Map:   0%|          | 0/599 [00:00<?, ? examples/s]

/users/Etu0/21402600/.local/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:3980: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


{'input': '[QUERY] Find information on President Barack Obama\'s family history, including genealogy, national origins, places and dates of birth, etc.\n[DOCUMENTS]\n[DOC 1] Family of Barack Obama - Wikipedia, the free encyclopedia\\nFamily of Barack Obama\\nFrom Wikipedia, the free encyclopedia\\n(Redirected from Sasha Obama)\\nJump to: navigation, search\\nObama Family\\nPresident Barack Obama, First Lady Michelle, and daughters Malia and Sasha wave to the crowd after his inaugural address Jan. 20, 2009, on the west steps of the U.S. Capitol.[1]\\nCurrent region\\nWashington, DC\\nInformation\\nPlace of origin\\nUnited States\\nNotable members\\nBarack Obama, Michelle Obama, Ann Dunham, Barack Obama, Sr., etc.\\nConnected families\\nRobinson, Dunham, Soetoro, Ng\\nThis article is part of a series about\\nBarack Obama\\nBackground \xa0ýý Illinois Senate \xa0ýý U.S. Senate\\nPolitical positions\xa0ýý Public image\xa0ýý Family\\n2008 primaries\xa0ýý ObamaýýýBiden campaign\\nTransition\x

In [ ]:
folds= generate_folds(tokenized_dataset, n_splits=5)

In [53]:
cross_val_train(modelname="t5-small", run_name="T5_SMALL_TOP_5_DOCS",filename="crossval_T5_SMALL_TOP5DOCS.jsonl", 
                tokenizer=tokenizer, tokenized_dataset=tokenized_dataset, 
                folds=folds, nb_epochs=30)

Processing Fold 1


All TF 2.0 model weights were used when initializing T5ForConditionalGeneration.

Some weights of T5ForConditionalGeneration were not initialized from the TF 2.0 model and are newly initialized: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipykernel_1981337/3761292519.py:53: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Bleu,Rouge1,Rouge2,Rougel,Meteor
1,24.110600,7.909437,0.047588,0.374766,0.157747,0.363523,0.275237
2,12.190700,1.011101,0.103273,0.386857,0.166034,0.377508,0.301269
3,2.538600,0.872277,0.114142,0.398919,0.178434,0.387180,0.319090
4,1.164700,0.828989,0.117945,0.409622,0.173707,0.393014,0.330334
5,1.164700,0.721032,0.135739,0.427961,0.195236,0.414530,0.358241
6,0.847300,0.673097,0.143557,0.433076,0.206606,0.422220,0.369737
7,0.747500,0.636529,0.148427,0.436883,0.212886,0.428279,0.378551
8,0.657900,0.590409,0.159183,0.450980,0.225189,0.442551,0.391678
9,0.657900,0.556572,0.167652,0.474435,0.229413,0.466709,0.418650
10,0.583200,0.544981,0.171319,0.479601,0.226465,0.472042,0.423932


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead u

Fold 1 results: {'eval_loss': 0.5113382935523987, 'eval_bleu': 0.1893099536860164, 'eval_rouge1': 0.4949540176339151, 'eval_rouge2': 0.24489164019572845, 'eval_rougeL': 0.4857354966359395, 'eval_meteor': 0.44466199798519823, 'eval_runtime': 1.6267, 'eval_samples_per_second': 73.77, 'eval_steps_per_second': 2.459, 'epoch': 30.0}


...

Fold 5 results: {'eval_loss': 0.47231248021125793, 'eval_bleu': 0.205260004896688, 'eval_rouge1': 0.49837764631903186, 'eval_rouge2': 0.23523881251568848, 'eval_rougeL': 0.4885402364015746, 'eval_meteor': 0.45092694757338664, 'eval_runtime': 1.7126, 'eval_samples_per_second': 69.486, 'eval_steps_per_second': 2.336, 'epoch': 30.0}


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 119/119 [00:00<00:00, 44648.20it/s]

Predictions for fold 5 saved to crossval_T5_SMALL_TOP5DOCS_fold_5.jsonl
Average metrics over all folds: {'eval_loss': 0.48240370750427247, 'eval_bleu': 0.19105834210343983, 'eval_rouge1': 0.48841880395297943, 'eval_rouge2': 0.2405301203580545, 'eval_rougeL': 0.48020857517432863, 'eval_meteor': 0.44378319799755195, 'eval_runtime': 1.8285399999999998, 'eval_samples_per_second': 66.708, 'eval_steps_per_second': 2.2276, 'epoch': 30.0}


Averages/epoch,▁
Averages/eval_bleu,▁
Averages/eval_loss,▁
Averages/eval_meteor,▁
Averages/eval_rouge1,▁
Averages/eval_rouge2,▁
Averages/eval_rougeL,▁
Averages/eval_runtime,▁
Averages/eval_samples_per_second,▁
Averages/eval_steps_per_second,▁
Fold_1/eval/epoch,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇████


Final predicitons for all folds foudn in crossval_T5_SMALL_TOP5DOCS.jsonl


# T5 SMALL----------------- TOP5 QUERY ONLY

In [9]:
tokenizer,tokenized_dataset= tokenize_and_split_dataset("FR_training_queryonly_qulac_PREPROCESSED_FOR_MODEL.json","t5-small")
folds= generate_folds(tokenized_dataset, n_splits=5)

Map:   0%|          | 0/596 [00:00<?, ? examples/s]

/users/Etu0/21402600/.local/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:3980: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [10]:
cross_val_train(modelname="t5-small", run_name="FR_T5_SMALL_QUERY_ONLY",filename="FR_crossval_T5_SMALL_QUERY_ONLY.jsonl", 
                tokenizer=tokenizer, tokenized_dataset=tokenized_dataset, 
                folds=folds, nb_epochs=30)

Processing Fold 1


W0000 00:00:1747094990.048300   62038 gpu_device.cc:2341] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
All TF 2.0 model weights were used when initializing T5ForConditionalGeneration.

Some weights of T5ForConditionalGeneration were not initialized from the TF 2.0 model and are newly initialized: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipykernel_62038/3145572364.py:53: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to th

Epoch,Training Loss,Validation Loss,Bleu,Rouge1,Rouge2,Rougel,Meteor
1,1.730600,1.043109,0.097236,0.352985,0.160334,0.347635,0.274443
2,1.022800,0.805679,0.106297,0.383997,0.171593,0.377809,0.316586
3,0.769100,0.668976,0.148166,0.439469,0.219237,0.428556,0.377078
4,0.691700,0.632927,0.160757,0.460511,0.240838,0.451540,0.408357
5,0.643600,0.617701,0.170637,0.464162,0.250486,0.455936,0.419589
6,0.633800,0.608882,0.179901,0.474578,0.256818,0.466017,0.429737
7,0.606700,0.602614,0.180354,0.482062,0.258170,0.472393,0.434164
8,0.582900,0.597470,0.176307,0.483201,0.253594,0.472649,0.435478
9,0.610300,0.593227,0.177802,0.492510,0.259799,0.481075,0.443780
10,0.563400,0.589826,0.183627,0.494818,0.265113,0.485611,0.450342


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].


Fold 1 results: {'eval_loss': 0.5771968960762024, 'eval_bleu': 0.21063748371946756, 'eval_rouge1': 0.5114530881982768, 'eval_rouge2': 0.2830265907342715, 'eval_rougeL': 0.5012040037233638, 'eval_meteor': 0.46832069446197144, 'eval_runtime': 2.1001, 'eval_samples_per_second': 57.139, 'eval_steps_per_second': 7.142, 'epoch': 29.0}


...

Fold 5 results: {'eval_loss': 0.5651502013206482, 'eval_bleu': 0.1984203652019145, 'eval_rouge1': 0.5086993385029102, 'eval_rouge2': 0.2782854137652523, 'eval_rougeL': 0.503345975184885, 'eval_meteor': 0.45837926472671403, 'eval_runtime': 2.4502, 'eval_samples_per_second': 48.567, 'eval_steps_per_second': 6.122, 'epoch': 30.0}


100%|██████████████████████████████████████████████████████████████████████| 119/119 [00:00<00:00, 99189.62it/s]

Predictions for fold 5 saved to FR_crossval_T5_SMALL_QUERY_ONLY_fold_5.jsonl
Average metrics over all folds: {'eval_loss': 0.5617120385169982, 'eval_bleu': 0.19748815657473706, 'eval_rouge1': 0.4994195036795254, 'eval_rouge2': 0.2690296099930828, 'eval_rougeL': 0.4912213500709476, 'eval_meteor': 0.4567228796827988, 'eval_runtime': 2.3174200000000003, 'eval_samples_per_second': 51.7918, 'eval_steps_per_second': 6.5162, 'epoch': 29.8}


Averages/epoch,▁
Averages/eval_bleu,▁
Averages/eval_loss,▁
Averages/eval_meteor,▁
Averages/eval_rouge1,▁
Averages/eval_rouge2,▁
Averages/eval_rougeL,▁
Averages/eval_runtime,▁
Averages/eval_samples_per_second,▁
Averages/eval_steps_per_second,▁
Fold_1/eval/epoch,▁▁▁▂▂▂▃▃▃▃▃▄▄▄▅▅▅▅▅▆▆▆▇▇▇▇▇███


Final predicitons for all folds foudn in FR_crossval_T5_SMALL_QUERY_ONLY.jsonl


# ---------- TOP 5 DOUCMENTS SUMMARIZED

In [12]:
tokenizer,tokenized_dataset= tokenize_and_split_dataset("training_top5_qulac_SUMMARIZED_FOR_MODEL.json","t5-small")
print(tokenized_dataset[0])  # Optional: check one example

Map:   0%|          | 0/599 [00:00<?, ? examples/s]

{'input': "[QUERY] Find information on President Barack Obama's family history, including genealogy, national origins, places and dates of birth, etc.\n[DOCUMENTS]\n[DOC 1] This page was last modified on 14 March 2009, at 05:50 . It contains articles with unsourced statements and articles needing factual verification since October 2008 .\n[DOC 2] This page was last modified on 14 March 2009, at 05:50 . It contains articles needing factual verification since October 2008 . It is available under the terms of the GNU Free Documentation License .\n[DOC 3] This page was last modified on 16 March 2009, at 03:22 . It contains articles with unsourced statements and articles needing factual verification since October 2008 . Hidden categories: Wikipedia semi-protected pages | All pages needing cleanup .\n[DOC 4] This page was last modified on 14 March 2009, at 05:50 . Use this page to help users with reading comprehension and vocabulary . Use the weekly Newsquiz to test your knowledge of stories

In [13]:
folds= generate_folds(tokenized_dataset, n_splits=5)

In [14]:
cross_val_train(modelname="t5-small", run_name="T5_SMALL_TOP5DOCS_SUMMARIZED",filename="crossval_T5_TOP5DOCS_SUMMARIZED.jsonl", 
                tokenizer=tokenizer, tokenized_dataset=tokenized_dataset, 
                folds=folds, nb_epochs=30)

Processing Fold 1


All TF 2.0 model weights were used when initializing T5ForConditionalGeneration.

Some weights of T5ForConditionalGeneration were not initialized from the TF 2.0 model and are newly initialized: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipykernel_14586/3145572364.py:53: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Bleu,Rouge1,Rouge2,Rougel,Meteor
1,1.393400,0.844550,0.098140,0.393720,0.172560,0.383675,0.300530
2,0.730600,0.601212,0.150760,0.431580,0.206855,0.424689,0.377058
3,0.586500,0.545290,0.170607,0.469422,0.220446,0.459851,0.417890
4,0.519100,0.530222,0.178776,0.470718,0.228101,0.461492,0.423201
5,0.491400,0.515488,0.182384,0.477691,0.228463,0.468648,0.429876
6,0.460800,0.508271,0.182371,0.478430,0.229796,0.469196,0.429252
7,0.438900,0.503539,0.185633,0.474441,0.231480,0.463858,0.425513
8,0.429800,0.499806,0.180491,0.475617,0.225040,0.467359,0.426904
9,0.432800,0.499436,0.186712,0.484756,0.230496,0.472845,0.436418
10,0.413600,0.499159,0.193403,0.480349,0.237614,0.470217,0.433564


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].


Fold 1 results: {'eval_loss': 0.4986312985420227, 'eval_bleu': 0.19074583115428184, 'eval_rouge1': 0.48904300069121504, 'eval_rouge2': 0.23124195063454211, 'eval_rougeL': 0.4744587812246184, 'eval_meteor': 0.43838852433833125, 'eval_runtime': 2.1723, 'eval_samples_per_second': 55.242, 'eval_steps_per_second': 6.905, 'epoch': 15.0}


...

Fold 5 results: {'eval_loss': 0.45843765139579773, 'eval_bleu': 0.20557456979612207, 'eval_rouge1': 0.5166019243725455, 'eval_rouge2': 0.25090219860397556, 'eval_rougeL': 0.5088385907134892, 'eval_meteor': 0.4691332159650356, 'eval_runtime': 2.0821, 'eval_samples_per_second': 57.155, 'eval_steps_per_second': 7.204, 'epoch': 17.0}


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 119/119 [00:00<00:00, 41414.05it/s]

Predictions for fold 5 saved to crossval_T5_TOP5DOCS_SUMMARIZED_fold_5.jsonl
Average metrics over all folds: {'eval_loss': 0.4672218382358551, 'eval_bleu': 0.19918105626307475, 'eval_rouge1': 0.4996960822925914, 'eval_rouge2': 0.251680731174653, 'eval_rougeL': 0.4900307651693466, 'eval_meteor': 0.45689439464484566, 'eval_runtime': 2.12884, 'eval_samples_per_second': 56.2888, 'eval_steps_per_second': 7.048, 'epoch': 15.8}


Averages/epoch,▁
Averages/eval_bleu,▁
Averages/eval_loss,▁
Averages/eval_meteor,▁
Averages/eval_rouge1,▁
Averages/eval_rouge2,▁
Averages/eval_rougeL,▁
Averages/eval_runtime,▁
Averages/eval_samples_per_second,▁
Averages/eval_steps_per_second,▁
Fold_1/eval/epoch,▁▁▂▃▃▃▄▅▅▅▆▇▇▇██


Final predicitons for all folds foudn in crossval_T5_TOP5DOCS_SUMMARIZED.jsonl


# HanDLE TFIDF WITH CROSS VAL

In [4]:
from sklearn.model_selection import KFold
from datasets import Dataset
from transformers import AutoTokenizer
import json
import numpy as np

#fucntion to tokenize the dataset based on the model
def tokenize_and_split_dataset(filename, modelname):
    with open(filename, "r", encoding="utf-8") as f:
        raw_data = json.load(f)
        
    # Choose your model
    tokenizer = AutoTokenizer.from_pretrained(modelname)

    # Tokenization function
    def preprocess(batch):
        model_inputs = tokenizer(
            batch["input"],
            max_length=512,
            padding="max_length",
            truncation=True
        )
        with tokenizer.as_target_tokenizer():
            labels = tokenizer(
                batch["output"],
                max_length=64,
                padding="max_length",
                truncation=True
            )
        model_inputs["labels"] = labels["input_ids"]
        return model_inputs
    
    # Convert raw data to Dataset object
    dataset = Dataset.from_list(raw_data)
    tokenized_dataset = dataset.map(preprocess, batched=True)

    return tokenizer, tokenized_dataset


In [4]:
import numpy as np
import json
def compute_metrics(eval_pred, tokenizer):
    """
    This function calculates BLEU, ROUGE, and METEOR for the model predictions.
    eval_pred: tuple (predictions, references), where predictions are model outputs
    and references are the true target labels.
    """
    predictions, labels = eval_pred
    predictions= predictions[0]
    pred_ids = np.argmax(predictions, axis=-1) 
    # Decode the model's predicted tokens and true labels
    predicted_texts = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    true_texts = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Calculate BLEU score
    bleu_score = bleu.compute(predictions=predicted_texts, references=true_texts)
    rouge_score = rouge.compute(predictions=predicted_texts, references=true_texts)
    meteor_score = meteor.compute(predictions=predicted_texts, references=true_texts)
   
    # Combine the metrics into a dictionary
    return {
        "bleu": bleu_score["bleu"],
        "rouge1": rouge_score["rouge1"],
        "rouge2": rouge_score["rouge2"],
        "rougeL": rouge_score["rougeL"],
        "meteor": meteor_score["meteor"]
    }




In [5]:
from sklearn.model_selection import KFold

#fucntion to split dataset into folds
def generate_folds(tokenized_dataset, n_splits=5, seed=42):
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=seed)
    folds = list(kf.split(tokenized_dataset))
    return folds


In [7]:
# recuperer les folds pour les indices de chaque fichier tfidf
tokenizer_global,tokenized_dataset_global= tokenize_and_split_dataset("FR_training_queryonly_qulac_PREPROCESSED_FOR_MODEL.json","t5-small")
folds= generate_folds(tokenized_dataset_global, n_splits=5)

Map:   0%|          | 0/596 [00:00<?, ? examples/s]

/users/Etu0/21402600/.local/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:3980: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [8]:
folds

[(array([  0,   1,   3,   4,   5,   7,   8,  12,  13,  14,  15,  16,  17,
          18,  19,  20,  21,  22,  23,  24,  25,  26,  27,  28,  31,  32,
          33,  34,  35,  36,  37,  38,  40,  41,  42,  43,  44,  45,  46,
          47,  48,  49,  50,  51,  52,  53,  54,  56,  57,  58,  59,  61,
          62,  64,  65,  66,  67,  68,  69,  71,  74,  75,  80,  85,  87,
          88,  89,  91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 102,
         103, 105, 106, 107, 108, 109, 111, 112, 113, 114, 115, 116, 117,
         119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 133,
         134, 136, 137, 138, 139, 141, 142, 143, 144, 146, 147, 149, 150,
         151, 152, 154, 155, 156, 157, 159, 160, 161, 162, 164, 166, 167,
         168, 169, 170, 171, 172, 173, 174, 175, 176, 178, 179, 180, 183,
         185, 186, 187, 189, 190, 191, 192, 193, 194, 195, 197, 198, 200,
         201, 202, 203, 205, 206, 207, 212, 213, 214, 215, 216, 217, 219,
         220, 221, 222, 223, 224, 225,

In [9]:
import wandb
from transformers import AutoModelForSeq2SeqLM, Trainer, TrainingArguments, EarlyStoppingCallback, TrainerCallback
from datasets import Dataset
import numpy as np
import json
import os
from tqdm import tqdm

# Custom callback to log metrics after each evaluation step
class WandbLoggingCallback(TrainerCallback):
    def __init__(self, fold_num):
        self.fold_num = fold_num

    def on_evaluate(self, args, state, control, metrics, **kwargs):
        if state.is_world_process_zero:
            log_data = {f"Fold_{self.fold_num}/eval/{key}": value for key, value in metrics.items()}
            wandb.log(log_data)

def cross_val_train(modelname, fold_filename, filename, run_name, tokenizer_global, tokenized_dataset_global, folds, nb_epochs=30):
    # Start a single WandB run for all folds
    wandb.init(project="cross_val_T5", name=run_name, config={"epochs": nb_epochs})
    filenamebis= filename.split(".jsonl")[0]
    fold_metrics = []

    for fold_num, (train_idx, test_idx) in enumerate(folds):
        print(f"Processing Fold {fold_num + 1}")
        
        #Ici on modifie pour recuperer le bon doc et appliquer le split dessus 3la 7ssab the fold
        file_tfdidf=fold_filename
        file_tfidf_fold=f"FOLD{fold_num}_{file_tfdidf}"
        tokenizer,tokenized_dataset= tokenize_and_split_dataset(file_tfidf_fold,modelname)
        print(f"Recuperation du Fichier {file_tfidf_fold} pour le training du Fold {fold_num}")
        
        # Split the dataset
        train_dataset = tokenized_dataset.select(train_idx)
        test_dataset = tokenized_dataset.select(test_idx)

        # Initialize the model
        model = AutoModelForSeq2SeqLM.from_pretrained(modelname, from_tf=True)

        # Training arguments with early stopping
        training_args = TrainingArguments(
            output_dir=f"./results/{run_name}/fold_{fold_num+1}",
            eval_strategy="epoch",
            save_strategy="epoch",
            learning_rate=5e-5,
            per_device_train_batch_size=8,
            per_device_eval_batch_size=8,
            num_train_epochs=nb_epochs,
            load_best_model_at_end=True,
            logging_steps=20,
            logging_first_step=True,
            metric_for_best_model="eval_loss",
            greater_is_better=False,
            seed=42,
        )

        # Initialize Trainer with the logging callback and early stopping
        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=train_dataset,
            eval_dataset=test_dataset,
            tokenizer=tokenizer,
            compute_metrics=lambda eval_pred: compute_metrics(eval_pred, tokenizer),
            callbacks=[
                WandbLoggingCallback(fold_num + 1),
                EarlyStoppingCallback(early_stopping_patience=3)
            ],
        )

        # Train the model
        trainer.train()

        # Final evaluation for the fold
        eval_results = trainer.evaluate()
        print(f"Fold {fold_num + 1} results: {eval_results}")

        # Store the final evaluation results for this fold
        fold_metrics.append(eval_results)

        # Predict on the test dataset
        predictions = trainer.predict(test_dataset)

    
        # Decode predictions and save to file
        pred_ids = np.argmax(predictions.predictions[0], axis=-1)
        predicted_texts = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
        true_texts = tokenizer.batch_decode(predictions.label_ids, skip_special_tokens=True)
        input_ids = test_dataset["input_ids"]
        input_texts = tokenizer.batch_decode(input_ids, skip_special_tokens=True)

        # Save predictions to a file

        output_predictions_file = f"{filenamebis}_fold_{fold_num+1}.jsonl"
        with open(output_predictions_file, "w", encoding="utf-8") as writer:
            for input_text, true_text, predicted_text in tqdm(zip(input_texts, true_texts, predicted_texts), total=len(true_texts)):
                output_dict = {
                    "input": input_text,
                    "true": true_text,
                    "predicted": predicted_text
                }
                writer.write(json.dumps(output_dict, ensure_ascii=False) + "\n")
        
        print(f"Predictions for fold {fold_num + 1} saved to {output_predictions_file}")

    # Calculate and log average metrics
    avg_metrics = {key: np.mean([fold[key] for fold in fold_metrics]) for key in fold_metrics[0]}
    wandb.log({f"Averages/{key}": value for key, value in avg_metrics.items()})
    print("Average metrics over all folds:", avg_metrics)

    wandb.finish()

    #merge all to a single file for predcitions

    with open(filename, "w", encoding="utf-8") as merged_file:
        for fold_num in range(len(folds)):
            fold_file = f"{filenamebis}_fold_{fold_num+1}.jsonl"
            if os.path.exists(fold_file):
                with open(fold_file, "r", encoding="utf-8") as f:
                    for line in f:
                        merged_file.write(line)
    print(f"Final predicitons for all folds foudn in {filename}")


# TEST TOP 5 DOCS TFDIDF

In [10]:
cross_val_train(modelname="t5-small", run_name="FR_T5_SMALL_TOP5DOCS_TFIDF",fold_filename="FR_training_top5_qulac_tfidf_CLEANED_rep_PREPROCESSED_FOR_MODEL.json",filename="FR_crossval_T5_SMALL_TOP5DOCS_TFIDF.jsonl", 
                tokenizer_global=tokenizer_global, tokenized_dataset_global=tokenized_dataset_global, 
                folds=folds, nb_epochs=25)

Processing Fold 1


Map:   0%|          | 0/596 [00:00<?, ? examples/s]

Recuperation du Fichier FOLD0_FR_training_top5_qulac_tfidf_CLEANED_rep_PREPROCESSED_FOR_MODEL.json pour le training du Fold 0


W0000 00:00:1747088260.959222   56665 gpu_device.cc:2341] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
All TF 2.0 model weights were used when initializing T5ForConditionalGeneration.

Some weights of T5ForConditionalGeneration were not initialized from the TF 2.0 model and are newly initialized: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipykernel_56665/3001149367.py:59: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to th

Epoch,Training Loss,Validation Loss,Bleu,Rouge1,Rouge2,Rougel,Meteor
1,1.438300,0.883603,0.111675,0.377948,0.172986,0.369415,0.301083
2,0.889600,0.715853,0.120112,0.410543,0.188912,0.401559,0.338239
3,0.690400,0.631655,0.162895,0.463888,0.232063,0.451113,0.395789
4,0.646200,0.614955,0.169162,0.472976,0.247314,0.460721,0.415384
5,0.599800,0.598372,0.170783,0.474768,0.249981,0.461383,0.424687
6,0.592600,0.589590,0.174162,0.483400,0.251815,0.468180,0.428656
7,0.570100,0.587388,0.180645,0.487023,0.261762,0.473955,0.438592
8,0.549500,0.583594,0.181274,0.485835,0.261197,0.473159,0.435626
9,0.570400,0.583231,0.186528,0.487300,0.263641,0.476203,0.437577
10,0.531800,0.581274,0.183078,0.487451,0.264657,0.476167,0.437838


...

Fold 5 results: {'eval_loss': 0.5550636649131775, 'eval_bleu': 0.19894933028567022, 'eval_rouge1': 0.5072599493564104, 'eval_rouge2': 0.27954840084580745, 'eval_rougeL': 0.4958626556724899, 'eval_meteor': 0.45486632453188813, 'eval_runtime': 2.5436, 'eval_samples_per_second': 46.784, 'eval_steps_per_second': 5.897, 'epoch': 25.0}


100%|██████████████████████████████████████████████████████████████████████| 119/119 [00:00<00:00, 55649.70it/s]

Predictions for fold 5 saved to FR_crossval_T5_SMALL_TOP5DOCS_TFIDF_fold_5.jsonl
Average metrics over all folds: {'eval_loss': 0.5529643297195435, 'eval_bleu': 0.19017669864614875, 'eval_rouge1': 0.49710817719617123, 'eval_rouge2': 0.2652272715041296, 'eval_rougeL': 0.4858846443963542, 'eval_meteor': 0.45209998767150344, 'eval_runtime': 2.27702, 'eval_samples_per_second': 52.791399999999996, 'eval_steps_per_second': 6.6424, 'epoch': 22.4}


Averages/epoch,▁
Averages/eval_bleu,▁
Averages/eval_loss,▁
Averages/eval_meteor,▁
Averages/eval_rouge1,▁
Averages/eval_rouge2,▁
Averages/eval_rougeL,▁
Averages/eval_runtime,▁
Averages/eval_samples_per_second,▁
Averages/eval_steps_per_second,▁
Fold_1/eval/epoch,▁▁▂▂▃▃▃▄▄▅▅▆▆▆▇▇███


Final predicitons for all folds foudn in FR_crossval_T5_SMALL_TOP5DOCS_TFIDF.jsonl


# TEST of top10 TFIDF INSTEAD OF TOP 5

In [10]:
cross_val_train(modelname="t5-small", run_name="FR_T5_SMALL_TOP10DOCS_TFIDF",fold_filename="FR_training_top10_qulac_tfidf_CLEANED_rep_PREPROCESSED_FOR_MODEL.json",filename="FR_crossval_T5_SMALL_TOP10DOCS_TFIDF.jsonl", 
                tokenizer_global=tokenizer_global, tokenized_dataset_global=tokenized_dataset_global, 
                folds=folds, nb_epochs=25)

Processing Fold 1


Map:   0%|          | 0/596 [00:00<?, ? examples/s]

Recuperation du Fichier FOLD0_FR_training_top10_qulac_tfidf_CLEANED_rep_PREPROCESSED_FOR_MODEL.json pour le training du Fold 0


W0000 00:00:1747091618.884770   59856 gpu_device.cc:2341] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
All TF 2.0 model weights were used when initializing T5ForConditionalGeneration.

Some weights of T5ForConditionalGeneration were not initialized from the TF 2.0 model and are newly initialized: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipykernel_59856/3001149367.py:59: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to th

Epoch,Training Loss,Validation Loss,Bleu,Rouge1,Rouge2,Rougel,Meteor
1,1.385900,0.861544,0.099598,0.378332,0.161224,0.367960,0.292824
2,0.891700,0.716012,0.124119,0.409322,0.191150,0.397628,0.336304
3,0.688400,0.633074,0.160094,0.457190,0.224685,0.442603,0.390003
4,0.649000,0.615146,0.166623,0.456563,0.234318,0.442833,0.401079
5,0.597600,0.597549,0.178528,0.477105,0.253367,0.463819,0.424821
6,0.592500,0.587435,0.175652,0.479105,0.253986,0.465509,0.425545
7,0.566100,0.586338,0.186651,0.480448,0.264036,0.466015,0.431776
8,0.544500,0.581283,0.188079,0.495036,0.264739,0.481592,0.441076
9,0.562600,0.579822,0.195121,0.497729,0.270666,0.485691,0.446504
10,0.528300,0.576138,0.192549,0.496250,0.273552,0.484572,0.445065


...

Fold 5 results: {'eval_loss': 0.5525848865509033, 'eval_bleu': 0.1962181489038196, 'eval_rouge1': 0.5141910217130199, 'eval_rouge2': 0.2798204956304613, 'eval_rougeL': 0.5045214978539461, 'eval_meteor': 0.4617980904015956, 'eval_runtime': 2.5434, 'eval_samples_per_second': 46.788, 'eval_steps_per_second': 5.898, 'epoch': 21.0}


100%|██████████████████████████████████████████████████████████████████████| 119/119 [00:00<00:00, 42287.74it/s]

Predictions for fold 5 saved to FR_crossval_T5_SMALL_TOP10DOCS_TFIDF_fold_5.jsonl
Average metrics over all folds: {'eval_loss': 0.5506801247596741, 'eval_bleu': 0.19386521343099197, 'eval_rouge1': 0.5022643172408191, 'eval_rouge2': 0.2704985020643252, 'eval_rougeL': 0.4924124377938914, 'eval_meteor': 0.4585384644361657, 'eval_runtime': 2.29124, 'eval_samples_per_second': 52.4658, 'eval_steps_per_second': 6.6015999999999995, 'epoch': 21.6}


Averages/epoch,▁
Averages/eval_bleu,▁
Averages/eval_loss,▁
Averages/eval_meteor,▁
Averages/eval_rouge1,▁
Averages/eval_rouge2,▁
Averages/eval_rougeL,▁
Averages/eval_runtime,▁
Averages/eval_samples_per_second,▁
Averages/eval_steps_per_second,▁
Fold_1/eval/epoch,▁▁▂▂▃▃▃▄▄▅▅▅▆▆▆▇▇███


Final predicitons for all folds foudn in FR_crossval_T5_SMALL_TOP10DOCS_TFIDF.jsonl
